# 네이버 매장정보 크롤링

#### 자료 
https://www.localdata.go.kr/devcenter/dataDown.do?menuNo=20001

1. 관광식당, 일반음식점, 휴게음식점 데이터셋 다운로드(07_24_01_P_.csv.zip, 07_24_04_P_.csv.zip , 07_24_05_P_.csv.zip)
2. 다운로드한 파일들을 data 폴더에 넣어주세요. (압축풀지말고)

문제 
1. 강남구 매장이지만, 네이버로 검색하는 경우 강남구가 아닌 다른 지역의 매장명이 검색됨
2. 데이터 반영이 빠른편이지만, 네이버에 매장이 없는 경우가 있음 (없는 매장 : 맥도날드, 교촌, 굽네) 
3. 검색하면 전혀 다른 업체가 나오는 경우가 있음 -> 네이버 업태구분명을 크롤링해와서 참고하여 반영해야 
4. 나라에서 제공하는 data를 사용하는 경우 인기도를 알기 어려움. 아닌가. -> 네이버 지도 api를 사용해서 점수를 부여할까? 
5. 자료 업데이트 시기 문제 - data를 다운로드 받아서 사용하는 경우, 매장 정보는 계속 바뀐는데 업데이트가 늦잖아... 매장 정보의 변동이 생기 사람이 계속 업로드를 해야 하잖아.. 

In [14]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

In [15]:
df = pd.read_csv('filtered_data_서울특별시강남구.csv')


In [16]:
df.head(5)

,영업상태명,소재지전체주소,도로명전체주소,사업장명,최종수정시점,데이터갱신일자,업태구분명,좌표정보(x),좌표정보(y),위생업태명,시도,지역구
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,2023-09-22 11:33:55,2023-09-24 00:17:57,한식,202203.971845,444853.761458,한식,서울특별시,강남구
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,2023-09-22 10:20:03,2023-09-24 00:17:57,한식,202462.794327,443827.961945,한식,서울특별시,강남구
2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,2023-12-01 17:58:46,2023-12-03 00:17:36,기타,205130.591679,445590.096838,기타,서울특별시,강남구
3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,2023-12-01 17:01:54,2023-12-03 00:17:36,한식,203470.848439,447369.579852,한식,서울특별시,강남구
4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,2023-12-01 16:01:58,2023-12-03 00:17:36,일식,204447.401189,445478.066611,일식,서울특별시,강남구


In [17]:
df.isna().sum()

영업상태명       0
소재지전체주소     0
도로명전체주소    95
사업장명        0
최종수정시점      0
데이터갱신일자     0
업태구분명       1
좌표정보(x)    81
좌표정보(y)    81
위생업태명       1
시도          0
지역구         0
dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16384 entries, 0 to 16383
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영업상태명    16384 non-null  object 
 1   소재지전체주소  16384 non-null  object 
 2   도로명전체주소  16289 non-null  object 
 3   사업장명     16384 non-null  object 
 4   최종수정시점   16384 non-null  object 
 5   데이터갱신일자  16384 non-null  object 
 6   업태구분명    16383 non-null  object 
 7   좌표정보(x)  16303 non-null  float64
 8   좌표정보(y)  16303 non-null  float64
 9   위생업태명    16383 non-null  object 
 10  시도       16384 non-null  object 
 11  지역구      16384 non-null  object 
dtypes: float64(2), object(10)
memory usage: 1.5+ MB


In [19]:
# 업태구분명 확인
unique_values = df['업태구분명'].unique()
unique_values

array(['한식', '기타', '일식', '분식', '호프/통닭', '중국식', '외국음식전문점(인도,태국등)', '경양식',
       '김밥(도시락)', '패밀리레스트랑', '식육(숯불구이)', '감성주점', '출장조리', '횟집', '라이브카페',
       '뷔페식', '정종/대포집/소주방', '복어취급', '까페', '통닭(치킨)', '탕류(보신용)', '패스트푸드',
       '냉면집', '키즈카페', '전통찻집', nan, '이동조리', '커피숍', '일반조리판매', '기타 휴게음식점',
       '편의점', '백화점', '아이스크림', '떡카페', '철도역구내', '푸드트럭', '과자점', '다방', '관광호텔'],
      dtype=object)

In [20]:
# 카페, 술, 출장요리 류 삭제
제외 = ['전통찻집','호프/통닭','뷔페식','출장조리','정종/대포집/소주방','이동조리', '감성주점','까페','라이브카페','키즈카페','커피숍','편의점', '일반조리판매','아이스크림', '떡카페', '철도역구내', '푸드트럭', '과자점', '다방', '관광호텔']
for i in 제외 :
    df = df.drop(df[df['업태구분명'] == i].index)
df

,영업상태명,소재지전체주소,도로명전체주소,사업장명,최종수정시점,데이터갱신일자,업태구분명,좌표정보(x),좌표정보(y),위생업태명,시도,지역구
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,2023-09-22 11:33:55,2023-09-24 00:17:57,한식,202203.971845,444853.761458,한식,서울특별시,강남구
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,2023-09-22 10:20:03,2023-09-24 00:17:57,한식,202462.794327,443827.961945,한식,서울특별시,강남구
2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,2023-12-01 17:58:46,2023-12-03 00:17:36,기타,205130.591679,445590.096838,기타,서울특별시,강남구
3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,2023-12-01 17:01:54,2023-12-03 00:17:36,한식,203470.848439,447369.579852,한식,서울특별시,강남구
4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,2023-12-01 16:01:58,2023-12-03 00:17:36,일식,204447.401189,445478.066611,일식,서울특별시,강남구
...,...,...,...,...,...,...,...,...,...,...,...,...
16350,영업/정상,서울특별시 강남구 삼성동 159-7번지 현대백화점,"서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)",심샤오빙,2019-02-11 12:33:10,2019-02-13 02:21:19,백화점,205210.358779,445154.422252,백화점,서울특별시,강남구
16355,영업/정상,서울특별시 강남구 대치동 316 은마상가 B블럭 지하1층 B-56호,"서울특별시 강남구 삼성로 212, 은마상가 B블럭동 지하1층 B-56호 (대치동, ...",미니 김밥,2022-07-04 15:34:15,2022-07-06 02:40:00,기타 휴게음식점,205707.089400,443914.194133,기타 휴게음식점,서울특별시,강남구
16358,영업/정상,서울특별시 강남구 논현동 242-52번지,"서울특별시 강남구 선릉로 657, 1층 103,105호 (논현동)",홍루이젠 강남구청역점,2019-01-31 17:43:36,2019-02-02 02:21:07,패스트푸드,203614.754292,445896.875886,패스트푸드,서울특별시,강남구
16367,영업/정상,서울특별시 강남구 대치동 912-23번지 우진빌딩,"서울특별시 강남구 삼성로 349, 우진빌딩 지하1층 (대치동)",G PC방 대치점,2019-01-22 17:16:39,2019-01-24 02:20:58,기타 휴게음식점,205101.289680,444497.311504,기타 휴게음식점,서울특별시,강남구


In [21]:
# 필요없는 칼럼 제거 

df.drop(['최종수정시점','데이터갱신일자','위생업태명'],axis=1,inplace=True)
df.head()

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,한식,202462.794327,443827.961945,서울특별시,강남구
2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,기타,205130.591679,445590.096838,서울특별시,강남구
3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구
4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,일식,204447.401189,445478.066611,서울특별시,강남구


In [22]:
# 도로명 주소만 추출

df['상세주소'] = df['도로명전체주소'].apply(lambda x: x.split(' ',4)[2] if pd.notna(x) else None) + df['도로명전체주소'].apply(lambda x: x.split(' ',4)[3] if pd.notna(x) else None)


# 셀레니움

- 원래 코드로 하면 계속 오류났음 

    -> ChromeDriverManager로 하면 오류나는 듯 해서 chromedriver_autoinstaller로 했습니다

In [23]:
df.head()

,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소
0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길18,"
1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로378,"
2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,기타,205130.591679,445590.096838,서울특별시,강남구,"영동대로513,"
3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로343,"
4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,일식,204447.401189,445478.066611,서울특별시,강남구,"봉은사로72길11,"


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12535 entries, 0 to 16380
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영업상태명    12535 non-null  object 
 1   소재지전체주소  12535 non-null  object 
 2   도로명전체주소  12475 non-null  object 
 3   사업장명     12535 non-null  object 
 4   업태구분명    12534 non-null  object 
 5   좌표정보(x)  12490 non-null  float64
 6   좌표정보(y)  12490 non-null  float64
 7   시도       12535 non-null  object 
 8   지역구      12535 non-null  object 
 9   상세주소     12475 non-null  object 
dtypes: float64(2), object(8)
memory usage: 1.1+ MB


In [25]:
## 20번 인덱스가 업음 ㄷㄷ 
df.reset_index(inplace=True)
df_200 = df.iloc[:200,:]

In [26]:
# # webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
# driver = webdriver.Chrome(ChromeDriverManager().install())
# # 주소 이동
# url = 'https://map.naver.com/'
# driver.get(url)
# time.sleep(1)


# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()
# 주소 이동
url = 'https://map.naver.com/'
driver.get(url)
time.sleep(1)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
df_200


,index,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,검색여부
0,0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길18,",가능
1,1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로378,",가능
2,2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,기타,205130.591679,445590.096838,서울특별시,강남구,"영동대로513,",불가능
3,3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로343,",가능
4,4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,일식,204447.401189,445478.066611,서울특별시,강남구,"봉은사로72길11,",불가능
...,...,...,...,...,...,...,...,...,...,...,...,...
195,200,영업/정상,서울특별시 강남구 도곡동 464 개포한신아파트,"서울특별시 강남구 언주로 123, 지상1층 101호 (도곡동, 개포한신아파트)",(주)일일향 도곡1호점,중국식,204254.782969,442744.847671,서울특별시,강남구,"언주로123,",불가능
196,202,영업/정상,서울특별시 강남구 논현동 114-20,"서울특별시 강남구 언주로134길 11, 지상1층 (논현동)",몽이네 한우내장탕,한식,203159.048808,446075.348000,서울특별시,강남구,"언주로134길11,",가능
197,203,영업/정상,서울특별시 강남구 역삼동 730-5,"서울특별시 강남구 언주로 405, 지상2층 202호 (역삼동)",밸런스드 볼(더짐 샐러드),기타,203814.127350,444111.900017,서울특별시,강남구,"언주로405,",불가능
198,204,영업/정상,서울특별시 강남구 역삼동 796-29 강남서해그랑블,"서울특별시 강남구 논현로63길 16, 지상1층 116호 (역삼동, 강남서해그랑블)",강남닭도리,한식,203389.562748,443422.779316,서울특별시,강남구,"논현로63길16,",가능


엑셀 파일엔 정보가 존재하는데 실제 식당이 없는경우 + 서브웨이처럼 이름은 같은데 지점만 다른 경우를 대비해서 도로명 주소 + 식당이름으로 크롤링함
 

In [ ]:
# 상세주소 + 사업장명으로 검색하는 코드 
# 그리고 식당이 존재하지 않는경우 검색결과가 없는데, 검색여부를 보고 실제 존재하는 식당만 추릴것임.
# 일단 10개만 ㄱㄱ 



from selenium.common.exceptions import NoSuchElementException



## 칼럼 생성 
df_200['검색여부'] = '모름'

for i in range(0,len(df_200.loc[:,['사업장명','상세주소']])):
    # 검색어를 네이버 url에 포함시켜 이동 + 검색 
    
    name = df_200['상세주소'][i] + ' ' +df_200['사업장명'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

    time.sleep(3)  

    try :
        searchIframe = driver.find_element(By.ID,'searchIframe')
        driver.switch_to.frame(searchIframe)
        time.sleep(3) 
        df_200['검색여부'][i] = '가능'
        print(name,'검색 완')
    except NoSuchElementException:
        print("식당 존재 x ", name)
        df_200['검색여부'][i] = '불가능'
        continue


강남대로114길18, 정식당 검색 완
강남대로378, 채선당 샤브보트 강남역점 검색 완
식당 존재 x  영동대로513, (주)푸들
압구정로343, 수아당 검색 완
식당 존재 x  봉은사로72길11, 오늘은 철판
역삼로204, 알찬한끼 검색 완
식당 존재 x  테헤란로77길41, 마스터즈
삼성로96길11, 세계관 검색 완
역삼로409, 분식을 품다 검색 완
식당 존재 x  언주로103길35, 곰푸드
식당 존재 x  언주로103길35, 곰프레시
식당 존재 x  언주로103길35, 바로푸드
논현로146길11, 식당 민홍 검색 완
식당 존재 x  논현로157길55, 디어(Dear)
식당 존재 x  개포로506, 버거리 개포동점
논현로8길11, 제주그집 검색 완
식당 존재 x  압구정로42길35, 카이센클럽(Kaisen Club)
식당 존재 x  선릉로152길7, 무(MUU)청담
강남대로628, 만수정 강남점 검색 완
식당 존재 x  강남대로124길39, 어웨이크닝
식당 존재 x  역삼로135, 찬스키친
언주로134길11, 은서네 국내산 대패삼겹살 검색 완
식당 존재 x  테헤란로517, 비비드크로넛 현대백화점 무역센터점
삼성로517, 냉동고집 삼성점 검색 완
역삼로220, 리하베스트 검색 완
논현로76길12, 팡뮤제 검색 완
논현로159길10, 만소당 검색 완
봉은사로74길19, 꼬모 서울 검색 완
식당 존재 x  봉은사로33길13, 논현브로스 심죽1호점
식당 존재 x  테헤란로517, 노티드 삼성 현대무역센터
논현로136길11, 겐지(KENJI) 검색 완
논현로136길5-5, 논현136 검색 완
압구정로36길7, 피양옥 검색 완
식당 존재 x  학동로12길21, 오덮밥
테헤란로6길30, 세븐어클락(7O'CLOCK) 검색 완
학동로4길8, 김재윤김치 검색 완
압구정로46길28, 앰비언트 피크닉(AMBIENT PICNIK) 검색 완
식당 존재 x  테헤란로322, 식연소
역삼로140, 쓰촨 마라 검색 완
논현로151길55, 스시 쥬고야 검색 완
역삼로140, 조선육회 

KeyError: 200

# 문제 1

식당 존재 x  압구정로343, 메밀집 압구정갤러리아점 -> 메밀집 갤러리아백화점 명품관 WEST점

식당 존재 x  언주로123, (주)일일향 도곡1호점 -> 언주로 123 일일향 도곡점

식당 존재 x  강남대로152길64, 이끼롤까스 가로수길본점 -> 서울 강남구 강남대로156길 35 1층

# 문제 2 
뻑남


실제 존재하는 식당만 추려냄 


In [ ]:
df_200

,index,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,검색여부
0,0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길18,",가능
1,1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로378,",가능
2,2,영업/정상,서울특별시 강남구 삼성동 159 코엑스,"서울특별시 강남구 영동대로 513, 코엑스 C홀 지상3층 (삼성동)",(주)푸들,기타,205130.591679,445590.096838,서울특별시,강남구,"영동대로513,",불가능
3,3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로343,",가능
4,4,영업/정상,서울특별시 강남구 삼성동 116-10,"서울특별시 강남구 봉은사로72길 11, 지하1층 (삼성동)",오늘은 철판,일식,204447.401189,445478.066611,서울특별시,강남구,"봉은사로72길11,",불가능
...,...,...,...,...,...,...,...,...,...,...,...,...
195,200,영업/정상,서울특별시 강남구 도곡동 464 개포한신아파트,"서울특별시 강남구 언주로 123, 지상1층 101호 (도곡동, 개포한신아파트)",(주)일일향 도곡1호점,중국식,204254.782969,442744.847671,서울특별시,강남구,"언주로123,",불가능
196,202,영업/정상,서울특별시 강남구 논현동 114-20,"서울특별시 강남구 언주로134길 11, 지상1층 (논현동)",몽이네 한우내장탕,한식,203159.048808,446075.348000,서울특별시,강남구,"언주로134길11,",가능
197,203,영업/정상,서울특별시 강남구 역삼동 730-5,"서울특별시 강남구 언주로 405, 지상2층 202호 (역삼동)",밸런스드 볼(더짐 샐러드),기타,203814.127350,444111.900017,서울특별시,강남구,"언주로405,",불가능
198,204,영업/정상,서울특별시 강남구 역삼동 796-29 강남서해그랑블,"서울특별시 강남구 논현로63길 16, 지상1층 116호 (역삼동, 강남서해그랑블)",강남닭도리,한식,203389.562748,443422.779316,서울특별시,강남구,"논현로63길16,",가능


In [ ]:
new_df_200 = df_200.loc[df_200['검색여부']=='가능',:]
new_df_200

,index,영업상태명,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),시도,지역구,상세주소,검색여부
0,0,영업/정상,서울특별시 강남구 논현동 198-9,"서울특별시 강남구 강남대로114길 18, 지상1층 102호 (논현동)",정식당,한식,202203.971845,444853.761458,서울특별시,강남구,"강남대로114길18,",가능
1,1,영업/정상,서울특별시 강남구 역삼동 825-9 준빌딩,"서울특별시 강남구 강남대로 378, 준빌딩 지상1층 (역삼동)",채선당 샤브보트 강남역점,한식,202462.794327,443827.961945,서울특별시,강남구,"강남대로378,",가능
3,3,영업/정상,서울특별시 강남구 압구정동 494 갤러리아백화점,"서울특별시 강남구 압구정로 343, 갤러리아백화점 지하1층 (압구정동)",수아당,한식,203470.848439,447369.579852,서울특별시,강남구,"압구정로343,",가능
5,5,영업/정상,서울특별시 강남구 역삼동 769-1 삼환 아르누보,"서울특별시 강남구 역삼로 204, 삼환 아르누보 지하1층 B09,B10호 중 107...",알찬한끼,분식,203453.074369,443736.685792,서울특별시,강남구,"역삼로204,",가능
7,7,영업/정상,서울특별시 강남구 삼성동 155-11,"서울특별시 강남구 삼성로96길 11, 지상2층 (삼성동)",세계관,한식,204952.006872,445195.026116,서울특별시,강남구,"삼성로96길11,",가능
...,...,...,...,...,...,...,...,...,...,...,...,...
191,195,영업/정상,서울특별시 강남구 역삼동 707-34 한신인터밸리24,"서울특별시 강남구 테헤란로 322, 한신인터밸리24 지하2층 제비145호 (역삼동)",캐주얼 샐러드,기타,204044.789949,444529.840427,서울특별시,강남구,"테헤란로322,",가능
193,197,영업/정상,서울특별시 강남구 역삼동 738-43 삼화빌딩,"서울특별시 강남구 논현로81길 13, 삼화빌딩 지상1층 (역삼동)",SPEVO 스페보,경양식,203188.832405,443960.976473,서울특별시,강남구,"논현로81길13,",가능
194,198,영업/정상,서울특별시 강남구 개포동 1188,"서울특별시 강남구 논현로8길 12, 지상1층 101호 (개포동)",13월,한식,204359.091172,441290.315272,서울특별시,강남구,"논현로8길12,",가능
196,202,영업/정상,서울특별시 강남구 논현동 114-20,"서울특별시 강남구 언주로134길 11, 지상1층 (논현동)",몽이네 한우내장탕,한식,203159.048808,446075.348000,서울특별시,강남구,"언주로134길11,",가능


카테고리 + 이름 없으면 ㄱㄱ 

In [ ]:
new_df_200.to_csv('new_df_200.csv')